In [1]:
import numpy as np
import tensorflow as tf
print("TF version:", tf.__version__)

# Import more tools
import tensorflow_hub as hub
print("TF Hub version:", hub.__version__)

# Check for GPU availability
print("GPU", "available 🤗" if tf.config.list_physical_devices("GPU") else "not available 😟")

TF version: 2.4.1
TF Hub version: 0.12.0
GPU not available 😟


In [2]:
engineering_set1 = np.load('/content/drive/MyDrive/Documents/Work/UOS Post-Doc/Amputee Posture Classification/1_pure_engineering_set.npy')
posture_classes1 = np.load('/content/drive/MyDrive/Documents/Work/UOS Post-Doc/Amputee Posture Classification/1_pure_engineering_set_classes.npy')
engineering_set2 = np.load('/content/drive/MyDrive/Documents/Work/UOS Post-Doc/Amputee Posture Classification/2_pure_engineering_set.npy')
posture_classes2 = np.load('/content/drive/MyDrive/Documents/Work/UOS Post-Doc/Amputee Posture Classification/2_pure_engineering_set_classes.npy')
engineering_set3 = np.load('/content/drive/MyDrive/Documents/Work/UOS Post-Doc/Amputee Posture Classification/3_pure_engineering_set.npy')
posture_classes3 = np.load('/content/drive/MyDrive/Documents/Work/UOS Post-Doc/Amputee Posture Classification/3_pure_engineering_set_classes.npy')

In [3]:
engineering_set = np.concatenate((engineering_set1, engineering_set2, engineering_set3), axis=0)
posture_classes = np.concatenate((posture_classes1, posture_classes2, posture_classes3), axis=0)

In [4]:
#ActivityCodes (0=sedentary 1=standing 2=stepping 2.1=cycling 3.1=primary lying, 3.2=secondary lying 4=non-wear 5=travelling)

def adjust_classes(classes):
  for count, value in enumerate(classes):
    if classes[count] == 2.1:
      classes[count] = 2
    elif classes[count] == 3.1:
      classes[count] = 3
    elif classes[count] == 3.2:
      classes[count] = 3
    elif classes[count] == 5.0:
      classes[count] = 0
    else:
      continue
  return classes 

posture_classes = adjust_classes(posture_classes)

In [5]:
# Removing non-wear because it destroys the NN
remove_non_wear_idx = posture_classes != 4
posture_classes = posture_classes[remove_non_wear_idx]
engineering_set = engineering_set[remove_non_wear_idx]

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(engineering_set, 
                                                    posture_classes,
                                                    test_size=0.2,
                                                    random_state=42)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

(157088, 295, 3)
(157088,)
(39272, 295, 3)
(39272,)


In [ ]:
train_shape = X_train.shape
test_shape = X_test.shape

new_shape_train = [1,295,3]
new_shape_train.insert(0, train_shape[0])
new_shape_train = tuple(new_shape_train)
new_shape_test = [1,295,3]
new_shape_test.insert(0, test_shape[0])
new_shape_test = tuple(new_shape_test)


X_train = X_train.reshape(new_shape_train)
X_test = X_test.reshape(new_shape_test)

In [7]:
# Find the unique label values...
unique_classes_train = np.unique(y_train)
unique_classes_test = np.unique(y_test)
y_train_hot = tf.one_hot(y_train, len(unique_classes_train))
y_test_hot = tf.one_hot(y_test, len(unique_classes_train))

# Count the unique label values
unique_train, counts_train = np.unique(y_train, return_counts=True)
unique_test, counts_test = np.unique(y_test, return_counts=True)
count_class_values_train = dict(zip(unique_train, counts_train))
count_class_values_test = dict(zip(unique_test, counts_test))

print(count_class_values_train)
print(count_class_values_test)

print(unique_classes_train)
print(unique_classes_test)
print(y_test)
print(y_test_hot)

print(y_train_hot.shape)
print(y_test_hot.shape)

{0.0: 54798, 1.0: 35903, 2.0: 11478, 3.0: 54909}
{0.0: 13744, 1.0: 8982, 2.0: 2876, 3.0: 13670}
[0. 1. 2. 3.]
[0. 1. 2. 3.]
[3. 1. 1. ... 1. 3. 0.]
tf.Tensor(
[[0. 0. 0. 1.]
 [0. 1. 0. 0.]
 [0. 1. 0. 0.]
 ...
 [0. 1. 0. 0.]
 [0. 0. 0. 1.]
 [1. 0. 0. 0.]], shape=(39272, 4), dtype=float32)
(157088, 4)
(39272, 4)


In [8]:
def norm_accel_data(x):
  """
  Processes accel values and returns a normalised integer (0-1).
  """
  x_minimum = 0
  x_maximum = 255
  x_normalized = ((x - x_minimum) / (x_maximum - x_minimum))
  return x_normalized

def process_epochs(x, y=None, test_data=False, shuffle=True):
  """
  Processing epochs
  """
  print("Creating training data...")
  # Normalize the acceleration data
  x = norm_accel_data(x)
  # Convert data to a tensor
  data = tf.constant(x)
  # If the data is a test dataset, just return the tensor
  if test_data:
    return data
  # If the data is a training dataset, we shuffle it
  if shuffle:
    indices = tf.range(start=0, limit=tf.shape(data)[0], dtype=tf.int32)
    shuffled_indices = tf.random.shuffle(indices)
    shuffled_x = tf.gather(x, shuffled_indices)
    shuffled_y = tf.gather(y, shuffled_indices)
    return shuffled_x, shuffled_y
  else:
    return data, y

In [9]:
# Create training and validation data batches
X_train_shuffled, y_train_hot_shuffled = process_epochs(X_train, y_train_hot, shuffle=False)
X_test_shuffled, y_test_hot_shuffled = process_epochs(X_test, y_test_hot, shuffle=False)

Creating training data...
Creating training data...


In [10]:
#INPUT_SHAPE = (1, 295, 3)
INPUT_SHAPE = (295, 3)
print('input_shape:', INPUT_SHAPE)

OUTPUT_SHAPE = len(unique_classes_train)
print('output_shape:', OUTPUT_SHAPE)

input_shape: (295, 3)
output_shape: 4


In [ ]:
# First basic model
model = tf.keras.Sequential()
model.add(tf.keras.Input(shape=(1, 295,3,)))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Dense(100, activation='relu'))
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(OUTPUT_SHAPE, activation='softmax'))
print(model.summary())

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

model.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

# Hyper-parameters
EPOCHS = 20

history = model.fit(X_train_shuffled,
                  y_train_hot_shuffled,
                  epochs=EPOCHS,
                  validation_split=0.2,
                  batch_size=32,
                  callbacks=[callback],
                  verbose=1)

import matplotlib.pyplot as plt

# summarize history for accuracy
plt.figure(figsize=(10, 4))
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.figure(figsize=(10, 4))
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# plot confusion matrix
from sklearn.metrics import classification_report
import sklearn.metrics as metrics
import seaborn as sns

LABELS = ['Sedentary', 'Standing', 'Stepping', 'Lying']

def show_confusion_matrix(validations, predictions):
    matrix = metrics.confusion_matrix(validations, predictions, normalize ='true')
    plt.figure(figsize=(6, 4))
    sns.heatmap(matrix,
                cmap='coolwarm',
                linecolor='white',
                linewidths=1,
                xticklabels=LABELS,
                yticklabels=LABELS,
                annot=True)
                #fmt='d')
    plt.title('Confusion Matrix')
    plt.ylabel('True Label')
    plt.xlabel('Predicted Label')
    plt.show()

# Print confusion matrix for training data
y_pred_test = model.predict(X_test_shuffled)
# Take the class with the highest probability from the train predictions
max_y_pred_test = np.argmax(y_pred_test, axis=1)
max_y_test_hot_shuffled = np.argmax(y_test_hot_shuffled, axis=1)

print('Unique Classes')
print(np.unique(max_y_pred_test))
print(np.unique(max_y_test_hot_shuffled))
print('------------')
show_confusion_matrix(max_y_test_hot_shuffled, max_y_pred_test)
print('------------')
print(classification_report(max_y_test_hot_shuffled, max_y_pred_test))

In [ ]:
# Model .2 CNN
model2 = tf.keras.Sequential([
                             tf.keras.layers.Conv2D(32, kernel_size = (1, 12),
                                                    strides = (1, 1),
                                                    input_shape = (1, 295, 3),
                                                    padding="valid"),
                             tf.keras.layers.MaxPooling2D((1,4), (1, 2)),
                              
                             tf.keras.layers.Conv2D(64, kernel_size = (1, 4),
                                                    strides = (1, 1),
                                                    input_shape = (1, 295, 3),
                                                    padding="valid"),
                             tf.keras.layers.MaxPooling2D((1,4), (1, 2)),
                              
                             tf.keras.layers.Flatten(),
                             tf.keras.layers.Dense(128, activation='relu'),
                             tf.keras.layers.Dense(OUTPUT_SHAPE, activation='softmax')
])

print(model2.summary())

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

model2.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

# Hyper-parameters
EPOCHS2 = 15

history2 = model2.fit(X_train_shuffled,
                  y_train_hot_shuffled,
                  epochs=EPOCHS2,
                  validation_split=0.2,
                  batch_size=32,
                  callbacks=[callback],
                  verbose=1)

# summarize history for accuracy
plt.figure(figsize=(10, 4))
plt.plot(history2.history['accuracy'])
plt.plot(history2.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.figure(figsize=(10, 4))
plt.plot(history2.history['loss'])
plt.plot(history2.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# Print confusion matrix for training data
y_pred_test2 = model2.predict(X_test_shuffled)
max_y_pred_test2 = np.argmax(y_pred_test2, axis=1)

print('Unique Classes')
print(np.unique(max_y_pred_test2))
print(np.unique(max_y_test_hot_shuffled))
print('------------')
show_confusion_matrix(max_y_test_hot_shuffled, max_y_pred_test2)
print('------------')
print(classification_report(max_y_test_hot_shuffled, max_y_pred_test2))

In [ ]:
# Model .3 LTSM

# The model
model3 = tf.keras.Sequential([
                              tf.keras.layers.LSTM(128, input_shape=(295, 3)),
                              tf.keras.layers.Dense(100, activation='relu'),
                              tf.keras.layers.Dense(OUTPUT_SHAPE, activation='softmax')
])

print(model3.summary())

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

model3.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

# Hyper-parameters
EPOCHS3 = 15

history3 = model3.fit(X_train_shuffled,
                  y_train_hot_shuffled,
                  epochs=EPOCHS3,
                  validation_split=0.2,
                  batch_size=32,
                  callbacks=[callback],
                  verbose=1)

# summarize history for accuracy
plt.figure(figsize=(10, 4))
plt.plot(history3.history['accuracy'])
plt.plot(history3.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.figure(figsize=(10, 4))
plt.plot(history3.history['loss'])
plt.plot(history3.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# plot confusion matrix
y_pred_test3 = model3.predict(X_test_shuffled)
max_y_pred_test3 = np.argmax(y_pred_test3, axis=1)

print('Unique Classes')
print(np.unique(max_y_pred_test3))
print(np.unique(max_y_test_hot_shuffled))
print('------------')
show_confusion_matrix(max_y_test_hot_shuffled, max_y_pred_test3)
print('------------')
print(classification_report(max_y_test_hot_shuffled, max_y_pred_test3))

In [ ]:
# Model .4 RNN

# The model
model4 = tf.keras.Sequential([
                              tf.keras.Input(shape=(295,3)),
                              tf.keras.layers.GRU(256, return_sequences=True),
                              tf.keras.layers.SimpleRNN(128),
                              tf.keras.layers.Dense(100, activation='relu'),
                              tf.keras.layers.Dense(OUTPUT_SHAPE, activation='softmax')
])

print(model4.summary())

callback = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=3)

model4.compile(loss='categorical_crossentropy',
              optimizer='adam', 
              metrics=['accuracy'])

# Hyper-parameters
EPOCHS4 = 15

history4 = model4.fit(X_train_shuffled,
                  y_train_hot_shuffled,
                  epochs=EPOCHS4,
                  validation_split=0.2,
                  batch_size=32,
                  callbacks=[callback],
                  verbose=1)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
gru (GRU)                    (None, 295, 256)          200448    
_________________________________________________________________
simple_rnn (SimpleRNN)       (None, 128)               49280     
_________________________________________________________________
dense (Dense)                (None, 100)               12900     
_________________________________________________________________
dense_1 (Dense)              (None, 4)                 404       
Total params: 263,032
Trainable params: 263,032
Non-trainable params: 0
_________________________________________________________________
None
Epoch 1/15
3928/3928 [==============================] - 3136s 798ms/step - loss: 0.8364 - accuracy: 0.6345 - val_loss: 0.4787 - val_accuracy: 0.8135
Epoch 2/15
  37/3928 [..............................] - ETA: 50:18 - loss: 0.4523 - accuracy: 

KeyboardInterrupt: ignored

In [ ]:
# summarize history for accuracy
plt.figure(figsize=(10, 4))
plt.plot(history4.history['accuracy'])
plt.plot(history4.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()
# summarize history for loss
plt.figure(figsize=(10, 4))
plt.plot(history4.history['loss'])
plt.plot(history4.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'validation'], loc='upper left')
plt.show()

# plot confusion matrix
y_pred_test4 = model4.predict(X_test_shuffled)
max_y_pred_test4 = np.argmax(y_pred_test4, axis=1)

print('Unique Classes')
print(np.unique(max_y_pred_test4))
print(np.unique(max_y_test_hot_shuffled))
print('------------')
show_confusion_matrix(max_y_test_hot_shuffled, max_y_pred_test4)
print('------------')
print(classification_report(max_y_test_hot_shuffled, max_y_pred_test4))